In [1]:
from pandas import DataFrame, read_csv
import pandas as pd
import numpy as np
from sklearn import linear_model
import math
from math import sqrt

In [2]:
kc_house=pd.read_csv(r'/Users/fengxiao/Documents/Coursera HM/Coursera_homework/WU Linear Regression/week1/kc_house_data.csv',index_col=False, header=0,dtype = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int})

In [3]:
train=pd.read_csv(r'/Users/fengxiao/Documents/Coursera HM/Coursera_homework/WU Linear Regression/week1/kc_house_train_data.csv',index_col=False, header=0,dtype = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int})

In [4]:
test=pd.read_csv(r'/Users/fengxiao/Documents/Coursera HM/Coursera_homework/WU Linear Regression/week1/kc_house_test_data.csv',index_col=False, header=0,dtype = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int})

Next write a function that takes a data set, a list of features (e.g. [‘sqft_living’, ‘bedrooms’]), to be used as inputs, and a name of the output (e.g. ‘price’). This function should return a features_matrix (2D array) consisting of first a column of ones followed by columns containing the values of the input features in the data set in the same order as the input list. It should also return an output_array which is an array of the values of the output in the data set (e.g. ‘price’). e.g. if you’re using SFrames and numpy you can complete the following function:

In [5]:
def get_numpy_data(data,feature,output):
    output=DataFrame(data,columns=output)
    output_array=output.values.reshape(1,len(data))
    data['intercept']=pd.Series(np.ones(len(data)))
    features=DataFrame(train,columns=feature)
    features_array=np.array(features)
    return(features_array,output_array)

In [11]:
(example_feature,example_output)=get_numpy_data(train,['intercept','sqft_living','bedrooms'],['price'])

 If the features matrix (including a column of 1s for the constant) is stored as a 2D array (or matrix) and the regression weights are stored as a 1D array then the predicted output is just the dot product between the features matrix and the weights (with the weights on the right). Write a function ‘predict_output’ which accepts a 2D array ‘feature_matrix’ and a 1D array ‘weights’ and returns a 1D array ‘predictions’. e.g. in python:

In [6]:
def predict_outcome(feature_matrix, weights):
    predictions=np.dot(feature_matrix,weights)
    return(predictions)

In [14]:
weights=np.ones(3)


In [15]:
prediction=predict_outcome(example_feature,weights)

If we have a the values of a single input feature in an array ‘feature’ and the prediction ‘errors’ (predictions - output) then the derivative of the regression cost function with respect to the weight of ‘feature’ is just twice the dot product between ‘feature’ and ‘errors’. Write a function that accepts a ‘feature’ array and ‘error’ array and returns the ‘derivative’ (a single number).


We are now going to move to computing the derivative of the regression cost function. Recall that the cost function is the sum over the data points of the squared difference between an observed output and a predicted output.
Since the derivative of a sum is the sum of the derivatives we can compute the derivative for a single data point and then sum over data points. We can write the squared difference between the observed output and predicted output for a single point as follows:
(w[0]*[CONSTANT] + w[1]*[feature_1] + ... + w[i] *[feature_i] + ... + w[k]*[feature_k] - output)^2
Where we have k features and a constant. So the derivative with respect to weight w[i] by the chain rule is:
2*(w[0]*[CONSTANT] + w[1]*[feature_1] + ... + w[i] *[feature_i] + ... + w[k]*[feature_k] - output)* [feature_i]
The term inside the paranethesis is just the error (difference between prediction and output). So we can re-write this as:
2*error*[feature_i]
That is, the derivative for the weight for feature i is the sum (over data points) of 2 times the product of the error and the feature itself. In the case of the constant then this is just twice the sum of the errors!
Recall that twice the sum of the product of two vectors is just twice the dot product of the two vectors. Therefore the derivative for the weight for feature_i is just two times the dot product between the values of feature_i and the current errors.
With this in mind complete the following derivative function which computes the derivative of the weight given the value of the feature (over all data points) and the errors (over all data points).

In [7]:
def feature_drivative(errors, feature):
    drivative=2*np.dot(errors,feature)
    return(drivative)

In [16]:
errors=prediction-example_output

In [17]:
feature_drivative(errors,example_feature)

array([[ -1.86802285e+10,  -4.71522256e+13,  -6.65680567e+10]])

Now we will use our predict_output and feature_derivative to write a gradient descent function. Although we can compute the derivative for all the features simultaneously (the gradient) we will explicitly loop over the features individually for simplicity. Write a gradient descent function that does the following:

Accepts a numpy feature_matrix 2D array, a 1D output array, an array of initial weights, a step size and a convergence tolerance.
While not converged updates each feature weight by subtracting the step size times the derivative for that feature given the current weights
At each step computes the magnitude/length of the gradient (square root of the sum of squared components)
When the magnitude of the gradient is smaller than the input tolerance returns the final weight vector.

In [49]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False
    weights = np.array(initial_weights)
    while not converged:
        # compute the predictions based on feature_matrix and weights:
        predictions = predict_outcome(feature_matrix,weights)
        # compute the errors as predictions - output:
        errors=predictions-output
        gradient_sum_squares = 0 # initialize the gradient
        # while not converged, update each weight individually:
        for i in range(len(weights)):
            # Recall that feature_matrix[:, i] is the feature column associated with weights[i]
            # compute the derivative for weight[i]:
            drivative=feature_drivative(errors,feature_matrix[:,i])
            
            # add the squared derivative to the gradient magnitude
            gradient_sum_squares+=drivative**2
            # update the weight based on step size and derivative:
            weights[i]-=step_size*drivative
        gradient_magnitude = sqrt(gradient_sum_squares)
        if gradient_magnitude < tolerance:
            converged = True
    return(weights)

In [50]:
simple_features = ['intercept','sqft_living']
my_output = ['price']
(simple_feature_matrix, output) = get_numpy_data(train, simple_features, my_output)
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

In [51]:
simple_weight=regression_gradient_descent(simple_feature_matrix,output,initial_weights,step_size,tolerance)

In [52]:
simple_weight

array([-46999.88716555,    281.91211918])

Now we will use the gradient descent to fit a model with more than 1 predictor variable (and an intercept). Use the following parameters:

model features = ‘sqft_living’, ‘sqft_living_15’
output = ‘price’
initial weights = [-100000, 1, 1] (intercept, sqft_living, and sqft_living_15 respectively)
step size = 4e-12
tolerance = 1e9

In [54]:
model_features = ['intercept','sqft_living', 'sqft_living15']
my_output = ['price']
(feature_matrix, output) = get_numpy_data(train, model_features,my_output)
initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9

In [55]:
multi_weight=regression_gradient_descent(feature_matrix,output,initial_weights,step_size,tolerance)

In [56]:
print multi_weight

[ -9.99999688e+04   2.45072603e+02   6.52795267e+01]


8.85263580285e+11


array([-46999.88764179,    280.63466326])